In [1]:
import pandas as pd
import os
import time
import fnmatch
import numpy as np
path = '/home/jxin05/Dataset/BB-MAS_Dataset/BB-MAS_Dataset/'
outpath = '/home/jxin05/BBMAS_Data/'
rawpath = '/home/jxin05/BBMAS_Data/rawdata/'
fixlen_path = '/home/jxin05/BBMAS_Data/fixlen_data/'

In [1]:
user_list = [str(i) for i in range(1,118)]

In [3]:
def find_checkpoint(path): 
    files = os.listdir(path)
    check1_exist = False
    check2_exist = False
    acc_data_exist = False
    check_sess1 = None
    check_sess2 = None
    acc_data = None
    for file in files:
        if(fnmatch.fnmatch(file, '*_HandPhone_Checkpoints_*')):
            check_sess1 = pd.read_csv(os.path.join(path + file))
            check1_exist = True
            #print("session 1 found for " + path)
        if(fnmatch.fnmatch(file, '*_HandTablet_Checkpoints_*')):
            check_sess2 = pd.read_csv(os.path.join(path + file))
            check2_exist = True
            #print("session 2 found for " + path)
        if(fnmatch.fnmatch(file, '*_PocketPhone_Accelerometer_*')):
            acc_data = pd.read_csv(os.path.join(path + file))
            acc_data_exist = True
            #print("acc data found for " + path)
    return check1_exist, check2_exist, acc_data_exist, check_sess1, check_sess2, acc_data

In [4]:
def check_valid(user_list, path):
    valid_user = []
    user_dict = {}
    for user in user_list:
        if user not in user_dict:
            user_dict[int(user)] = {}
        check1_exist, check2_exist, acc_data_exist, check_sess1, check_sess2, acc_data = find_checkpoint(path+user+'/')
        if(check1_exist and check2_exist and acc_data_exist):
            valid_user.append(user)
            user_dict[int(user)]['check1'] = check_sess1
            user_dict[int(user)]['check2'] = check_sess2
            user_dict[int(user)]['acc_data'] = acc_data
    return valid_user, user_dict

In [5]:
valid_user, user_dict = check_valid(user_list, path)

In [6]:
def get_start_end(check_sess1, check_sess2):
    start1, start2, end1, end2 = 0,0,0,0
    for idx in range(len(check_sess1)):
        row = check_sess1.iloc[idx]
        if(row['EID'] == 1):
            start1 = row['time']
        if(row['EID'] == 14):
            end1 = row['time']
    for idx in range(len(check_sess2)):
        row = check_sess2.iloc[idx]
        if(row['EID'] == 1):
            start2 = row['time']
        if(row['EID'] == 14):
            end2 = row['time']
    if(start1 == 0 or end1 == 0):
        print("incomplete data ins session 1")
    if(start2 == 0 or end2 == 0):
        print("incomplete data ins session 2")
    return start1,end1,start2, end2

In [7]:
def make_time_col(df):
    df['time_in_ms'] = df['time'].apply(make_time)
    return df
def make_time(timestamp):
    return time.mktime(time.strptime(timestamp[:-4],'%Y-%m-%d %H:%M:%S')) +float('0'+timestamp[-4:])

In [8]:
def find_index(df, start1, end1, start2, end2):
    if(start1 == 0 or end1 == 0 or start2 == 0 or end2 == 0):
        print('some data is missing, do not use this user')
        return
    start1 = make_time(start1)
    start2 = make_time(start2)
    end1 = make_time(end1)
    end2 = make_time(end2)
    df = make_time_col(df)
    start_idx1, start_idx2, end_idx1, end_idx2 = -1,-1,-1,-1
    for idx in range(len(df)):
        row = df.iloc[idx]
        if(row['time_in_ms'] >= start1 and row['time_in_ms'] - start1 <= 5):
            start_idx1 = idx
        if(row['time_in_ms'] >= start2 and row['time_in_ms'] - start2 <= 5):
            start_idx2 = idx
        if(row['time_in_ms'] <= end1 and end1 - row['time_in_ms'] <= 5):
            end_idx1 = idx
        if(row['time_in_ms'] <= end2 and end2 - row['time_in_ms'] <= 5):
            end_idx2 = idx
    if(start_idx1 > 0 and end_idx1 > 0 and start_idx2 > 0 and end_idx2 > 0):
        return start_idx1, end_idx1, start_idx2, end_idx2
    else:
        print('did not find correct index')
        return

In [9]:
def split_session(outpath, user_dict):
    for user in user_dict.keys():
        if(user_dict[user] == {}):
            continue
        print("processing data from user" + str(user))
        start1,end1,start2, end2 = get_start_end(user_dict[user]['check1'], user_dict[user]['check2'])
        if(start1 == 0 or end1 == 0 or start2 == 0 or end2 == 0):
            print('user' + str(user) + " has missing data")
            continue
        try:
            start_idx1, end_idx1, start_idx2, end_idx2 = find_index(user_dict[user]['acc_data'], start1,end1,start2, end2)
        except:
            print('stop using user ' + str(user))
            continue
        sess1 = user_dict[user]['acc_data'].iloc[start_idx1:end_idx1+1]
        sess2 = user_dict[user]['acc_data'].iloc[start_idx2:end_idx2+1]
        if not os.path.exists(outpath):
            os.makedirs(outpath)
        sess1.to_csv(outpath + str(user) + 'session1.csv',index=False)
        sess2.to_csv(outpath + str(user) + 'session2.csv',index=False)
        

In [10]:
split_session(rawpath, user_dict)

processing data from user10
processing data from user11
processing data from user13
processing data from user15
processing data from user16
processing data from user17
processing data from user18
processing data from user19


In [11]:
def remove_col(df):
    return df.drop(['EID','time','time_in_ms'], axis =1)

In [12]:
def fix_len(length, outpath,inpath = rawpath):
    files = os.listdir(inpath)
    for file in files:
        df = pd.read_csv(inpath + file)
        if len(df) <= length:
            continue
        diff = int((len(df) - length)/2)
        processed = df.iloc[diff:diff+length]
        processed.to_csv(outpath+file, index = False)

In [27]:
fix_len(12000, fixlen_path)

In [17]:
def mix_data(truepath, fake_number, replace_time, replace_size = 200):
    file_list = os.listdir(truepath)
    pick_dict = {}
    mixed_data = []
    count = 0
    
    while(count < fake_number):
        pick = np.random.randint(0,len(file_list))
        pick_dict[count] = []
        print("creating fake sample from " + file_list[pick])
        target = pd.read_csv(truepath + file_list[pick])
        mix = np.asarray(remove_col(target).values)
        for j in range(replace_time): 
            start = np.random.randint(3,len(target)/replace_size-1)
            while(start in pick_dict[count]):
                start = np.random.randint(3,len(target)/replace_size-1)
            pick_dict[count].append(start)
            replace_clip = take_from_others(truepath, replace_size, pick, file_list)
            mix[start*replace_size:(start+1)*replace_size, :] = replace_clip
        mixed_data.append(mix)
        print("clips replaced: {}".format(np.sort(pick_dict[count])))
        count+=1
        print("**********")
    return mixed_data, pick_dict           
    

In [18]:
def take_from_others(truepath, replace_size, pick, file_list):
    takefrom = np.random.randint(0,len(file_list))
    while(takefrom == pick):
        takefrom = np.random.randint(0,len(file_list))
    target_df = pd.read_csv(truepath+file_list[takefrom])
    start_idx = np.random.randint(0,len(target_df)-replace_size)
    replace_clip = target_df.iloc[start_idx:start_idx+replace_size]
    replace_clip = remove_col(replace_clip)
    return np.asarray(replace_clip.values)

In [136]:
mixed_data1, pick_dict1 = mix_data(fixlen_path, 2, 10) 


creating fake sample from 34session1.csv
clips replaced: [11 20 24 27 33 35 41 46 49 50]
**********
creating fake sample from 27session1.csv
clips replaced: [ 5 14 21 28 29 35 37 43 49 53]
**********


In [141]:
def smooth(mixed_data, pick_dict, window = 3):
    res = mixed_data.copy()
    for i in range(len(mixed_data)):
        for v in pick_dict[i]:
            #print(v)
            start = v*200-10
            end = v*200+10
            data_x = mixed_data[i][start:end,0].copy()
            #print(data_x)
            data_y = mixed_data[i][start:end,1].copy()
            data_z = mixed_data[i][start:end,2].copy()
            smoothed_x = ExpMovingAverage(data_x, window)
            #print(smoothed_x)
            smoothed_y = ExpMovingAverage(data_y, window)
            smoothed_z = ExpMovingAverage(data_z, window)
            res[i][start+window+1:end-window,0] = smoothed_x[window+1:-window]
            #print(res[i][start+window+1:end-window,0])
            res[i][start+window+1:end-window,1] = smoothed_y[window+1:-window]
            res[i][start+window+1:end-window,2] = smoothed_z[window+1:-window]
    return res

In [140]:
def ExpMovingAverage(array, window):
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    
    a = np.convolve(array, weights, mode='full')[:len(array)]
    a[:window] = a[window]
    return a

In [31]:
def make_fake_dataset(mixed_data, pick_dict):
    sample_len = mixed_data[0].shape[0]
    num_sample = len(mixed_data)
    X = np.zeros((num_sample, mixed_data[0].shape[1], sample_len))
    Y = np.zeros((num_sample, 60))
    for i in range(len(mixed_data)):
        X[i,:,:] = mixed_data[i].T
        for v in pick_dict[i]:
            Y[i,v] = 1
    return X,Y

In [32]:
X,Y = make_fake_dataset(mixed_data1, pick_dict1)